In [1]:
import pandas as pd
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

# for modeling
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings("ignore")

Reading the dataset

In [2]:
dataset = pd.read_csv('digits.csv')
dataset.columns=['d1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'd11', 'd12', 'd13', 'd14', 'd15', 'd16', 'output']
dataset.head(5)

,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,output
0,80,100,18,98,60,66,100,29,42,0,0,23,42,61,56,98,8
1,0,94,9,57,20,19,7,0,20,36,70,68,100,100,18,92,8
2,95,82,71,100,27,77,77,73,100,80,93,42,56,13,0,0,9
3,68,100,6,88,47,75,87,82,85,56,100,29,75,6,0,0,9
4,70,100,100,97,70,81,45,65,30,49,20,33,0,16,0,0,1


Train-test split

In [3]:
train, test = train_test_split(dataset, train_size=0.8, random_state=8)
test.reset_index(drop=True)
xtrain = train.iloc[:, 0:-1]
ytrain = train.iloc[:,-1]
xtest = test.iloc[:, 0:-1]
ytest = test.iloc[:,-1]

## Neural network

Building the framework

In [4]:
model = Sequential()
model.add(Dense(16, input_shape=(xtrain.shape[1],), activation='relu')) 
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 128)               2176      
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 20,250
Trainable params: 20,250
Non-trainable params: 0
_________________________________________________________________


Defining the optimization algorithm

In [5]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

Define early stopping method

In [6]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                   mode='min',
                                   patience=5, 
                                   restore_best_weights=True)

Encoding the output using one-hot encoding

In [7]:
Y_train_onehot = tf.keras.utils.to_categorical(ytrain)
Y_test_onehot = tf.keras.utils.to_categorical(ytest)

Fitting the algorithm

In [8]:
yhistory = model.fit(xtrain,
                    Y_train_onehot,
                    callbacks=[es],
                    epochs=20, 
                    batch_size=10,
                    shuffle=True,
                    validation_split=0.2,
                    verbose=1)

Epoch 1/20
704/704 [==============================] - 2s 2ms/step - loss: 0.6995 - accuracy: 0.8528 - val_loss: 0.4072 - val_accuracy: 0.9096
Epoch 2/20
704/704 [==============================] - 1s 2ms/step - loss: 0.2118 - accuracy: 0.9400 - val_loss: 0.1479 - val_accuracy: 0.9665
Epoch 3/20
704/704 [==============================] - 1s 1ms/step - loss: 0.1195 - accuracy: 0.9676 - val_loss: 0.1578 - val_accuracy: 0.9557
Epoch 4/20
704/704 [==============================] - 1s 1ms/step - loss: 0.1021 - accuracy: 0.9701 - val_loss: 0.0906 - val_accuracy: 0.9756
Epoch 5/20
704/704 [==============================] - 1s 1ms/step - loss: 0.0861 - accuracy: 0.9758 - val_loss: 0.1883 - val_accuracy: 0.9426
Epoch 6/20
704/704 [==============================] - 1s 1ms/step - loss: 0.0756 - accuracy: 0.9765 - val_loss: 0.0996 - val_accuracy: 0.9750
Epoch 7/20
704/704 [==============================] - 1s 1ms/step - loss: 0.0602 - accuracy: 0.9827 - val_loss: 0.0772 - val_accuracy: 0.9829
Epoch 

Evaluate the prediction accuracy

In [9]:
y_preds = model.predict(xtest)
print((y_preds.argmax(axis=-1) == ytest).sum() / len(xtest))

69/69 [==============================] - 0s 915us/step
0.992269213278763
